In [2]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [4]:
# to reload
%dotenv

In [3]:
from google.cloud import bigquery

client = bigquery.Client()

In [60]:
p_sample = pd.read_excel("../projects/gain_tracker/tests/positions_sample.xlsx")

In [44]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price           float64
position_exit_date      datetime64[ns]
position_exit_price            float64
recognized_gain                float64
dtype: object

### change to Decimals

In [1]:
from decimal import Decimal

In [ ]:
p_sample.loc[:, "position_entry_price"] = p_sample["position_entry_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

p_sample.loc[:, "position_exit_price"] = p_sample["position_exit_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

In [52]:
p_sample.tail()

,portfolio_id,position_id,external_id,id_type,position_entry_date,exchange,quantity,status,position_entry_price,position_exit_date,position_exit_price,recognized_gain
26,Roth IRA,26,TSLA,TICKER,2024-01-25,PCX,2,OPEN_LONG,180.4,NaT,NaN,NaN
27,IRA,27,DQ,TICKER,2024-01-25,PCX,20,OPEN_LONG,19.5,NaT,NaN,NaN
28,IRA,28,CVS,TICKER,2024-01-25,PCX,8,OPEN_LONG,71.99,NaT,NaN,NaN
29,Roth IRA,29,BLDE,TICKER,2024-01-30,PCX,160,OPEN_LONG,3.0,NaT,NaN,NaN
30,IRA,30,GPS,TICKER,2024-01-31,PCX,20,OPEN_LONG,18.9,NaT,NaN,NaN


In [53]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price            object
position_exit_date      datetime64[ns]
position_exit_price             object
recognized_gain                float64
dtype: object

In [54]:
job = client.load_table_from_dataframe(
    p_sample, "main-street-labs.gain_tracker_dev.positions_dec"
)

In [56]:
# Perform a query.
QUERY = (
    'SELECT position_id, position_entry_price, position_exit_price FROM `main-street-labs.gain_tracker_dev.positions_dec`'
)
p_dec_sample_dl = client.query_and_wait(QUERY).to_dataframe()  # API request

In [108]:
p_dec_sample_dl.tail()

,position_id,position_entry_price,position_exit_price
26,1,156.000,158.0000
27,0,210.130,470.0000
28,4,7.895,9.4000
29,5,7.895,9.2349
30,3,39.030,46.0700


In [5]:
# Perform a query.
QUERY = (
    'SELECT * FROM `main-street-labs.gain_tracker_dev.etrade_positions`'
)
etrade_pos = client.query_and_wait(QUERY).to_dataframe()  # API request

In [7]:
etrade_pos_2 = client.query_and_wait(QUERY).to_dataframe().set_index("POSITIONID")

In [10]:
etrade_pos.set_index("POSITIONID", inplace=True)
etrade_pos.head()

,SYMBOLDESCRIPTION,DATEACQUIRED,PRICEPAID,COMMISSIONS,OTHERFEES,QUANTITY,POSITIONINDICATOR,POSITIONTYPE,DAYSGAIN,DAYSGAINPCT,...,TODAYFEES,TODAYPRICEPAID,TODAYQUANTITY,ADJPREVCLOSE,LOTSDETAILS,QUOTEDETAILS,PRODUCT,QUICK,ACCOUNT_ID_KEY,__index_level_0__
POSITIONID,,,,,,,,,,,,,,,,,,,,,
93444125900,BARDX,1598414400000,9.0643,0.0,0,242.6009,TYPE1,LONG,-4.8520,-0.2205,...,0,0,0,9.07,https://api.etrade.com/v1/accounts/lP_HmzzVyiL...,https://api.etrade.com/v1/market/quote/BARDX.json,"{'expiryDay': 0, 'expiryMonth': 0, 'expiryYear...","{'change': -0.02, 'changePct': -0.2205, 'lastT...",lP_HmzzVyiLFftb7DXaEUw,9
41190282005,PORTX,1481864400000,38.0497,0.0,0,74.1910,TYPE1,LONG,17.8058,0.4176,...,0,0,0,57.46,https://api.etrade.com/v1/accounts/lP_HmzzVyiL...,https://api.etrade.com/v1/market/quote/PORTX.json,"{'expiryDay': 0, 'expiryMonth': 0, 'expiryYear...","{'change': 0.24, 'changePct': 0.4176, 'lastTra...",lP_HmzzVyiLFftb7DXaEUw,2
349212557900,LI,1700629200000,38.6432,0.0,0,40.0000,TYPE1,LONG,-27.6000,-2.2294,...,0,0,0,30.95,https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,https://api.etrade.com/v1/market/quote/LI.json,"{'expiryDay': 0, 'expiryMonth': 0, 'expiryYear...","{'change': -0.69, 'changePct': -2.2294, 'lastT...",E2ayfoKSzH1fFZs-7xV48A,2
359270893900,AMD,1705986000000,166.1999,0.0,0,5.0000,TYPE1,LONG,15.3000,1.8227,...,0,0,0,167.88,https://api.etrade.com/v1/accounts/SXPtmpM7IZI...,https://api.etrade.com/v1/market/quote/AMD.json,"{'expiryDay': 0, 'expiryMonth': 0, 'expiryYear...","{'change': 3.06, 'changePct': 1.8227, 'lastTra...",SXPtmpM7IZIpymUjhjI-zg,9
337469285900,ADTN,1694404800000,7.2654,0.0,0,110.0000,TYPE1,LONG,-19.8000,-2.7439,...,0,0,0,6.56,https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,https://api.etrade.com/v1/market/quote/ADTN.json,"{'expiryDay': 0, 'expiryMonth': 0, 'expiryYear...","{'change': -0.18, 'changePct': -2.7439, 'lastT...",E2ayfoKSzH1fFZs-7xV48A,17


In [15]:
len(etrade_pos)

51

In [16]:
pos_diff = etrade_pos.loc[~etrade_pos.index.isin(etrade_pos_2.head(10).index)]

In [17]:
len(pos_diff)

41

# Use ETrade API

In [4]:
import pandas as pd
from datetime import date, datetime

import os
import sys
sys.path.append("../projects/gain_tracker")

In [5]:
from gain_tracker.resources.etrade_api import ETradeAPI

In [35]:
datetime.fromtimestamp(1704258000000/1000).date().isoformat()

'2024-01-03'

In [6]:
# consumer_key = os.getenv("ETRADE_SANDBOX_KEY")
# consumer_secret = os.getenv("ETRADE_SANDBOX_SECRET")
consumer_key = os.getenv("ETRADE_KEY")
consumer_secret = os.getenv("ETRADE_SECRET")

In [7]:
etrader = ETradeAPI("prod")

In [25]:
etrader.authenticate_session()

Please accept agreement and enter verification code from browser:  E65XK


accounts = etrader.list_accounts()

In [61]:
len(accounts)

3

etrader.session_token

etrader.session_token_secret

In [29]:
a_etrader = ETradeAPI("prod", session_token=etrader.session_token, session_token_secret=etrader.session_token_secret)

In [30]:
session = a_etrader.create_authenticated_session()

In [31]:
response = session.get("https://api.etrade.com/oauth/renew_access_token")
response.status_code

200

In [31]:
# list_accounts_url = 'https://apisb.etrade.com/v1/accounts/list.json'
list_accounts_url = 'https://api.etrade.com/v1/accounts/list.json'

In [33]:
resp.json()["AccountListResponse"]["Accounts"]["Account"]

[{'accountId': '153285977',
  'accountIdKey': 'lP_HmzzVyiLFftb7DXaEUw',
  'accountMode': 'CASH',
  'accountDesc': 'Individual Brokerage',
  'accountName': '',
  'accountType': 'INDIVIDUAL',
  'institutionType': 'BROKERAGE',
  'accountStatus': 'ACTIVE',
  'closedDate': 0,
  'shareWorksAccount': False,
  'fcManagedMssbClosedAccount': False},
 {'accountId': '227699084',
  'accountIdKey': 'E2ayfoKSzH1fFZs-7xV48A',
  'accountMode': 'CASH',
  'accountDesc': 'Roth IRA',
  'accountName': '',
  'accountType': 'ROTHIRA',
  'institutionType': 'BROKERAGE',
  'accountStatus': 'ACTIVE',
  'closedDate': 0,
  'shareWorksAccount': False,
  'fcManagedMssbClosedAccount': False},
 {'accountId': '892405345',
  'accountIdKey': 'SXPtmpM7IZIpymUjhjI-zg',
  'accountMode': 'CASH',
  'accountDesc': 'Traditional IRA',
  'accountName': '',
  'accountType': 'CONTRIBUTORY',
  'institutionType': 'BROKERAGE',
  'accountStatus': 'ACTIVE',
  'closedDate': 0,
  'shareWorksAccount': False,
  'fcManagedMssbClosedAccount': 

In [32]:
resp = session.get(list_accounts_url, params = {'format': 'json'})

accounts = pd.DataFrame(resp.json()["AccountListResponse"]["Accounts"]["Account"])

accounts

,accountId,accountIdKey,accountMode,accountDesc,accountName,accountType,institutionType,accountStatus,closedDate,shareWorksAccount,fcManagedMssbClosedAccount
0,153285977,lP_HmzzVyiLFftb7DXaEUw,CASH,Individual Brokerage,,INDIVIDUAL,BROKERAGE,ACTIVE,0,False,False
1,227699084,E2ayfoKSzH1fFZs-7xV48A,CASH,Roth IRA,,ROTHIRA,BROKERAGE,ACTIVE,0,False,False
2,892405345,SXPtmpM7IZIpymUjhjI-zg,CASH,Traditional IRA,,CONTRIBUTORY,BROKERAGE,ACTIVE,0,False,False


In [99]:
accountIdKey = "SXPtmpM7IZIpymUjhjI-zg"
accountIdKey = "E2ayfoKSzH1fFZs-7xV48A"
instType = "BROKERAGE"

In [39]:
account_balances_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/balance.json"

In [75]:
account_balances_url

'https://apisb.etrade.com/v1/accounts/xj1Dc18FTqWPqkEEVUr5rw/balance'

In [76]:
resp = session.get(account_balances_url, params = {"instType":instType,"realNAV": False})

In [77]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><BalanceResponse><accountId>833535350</accountId><accountType>MARGIN</accountType><optionLevel>LEVEL_4</optionLevel><accountDescription></accountDescription><Cash><fundsForOpenOrdersCash>0.0</fundsForOpenOrdersCash><moneyMktBalance>0.0</moneyMktBalance></Cash><Computed><cashAvailableForInvestment>0.00</cashAvailableForInvestment><cashAvailableForWithdrawal>0.00</cashAvailableForWithdrawal><netCash>-740.450013</netCash><cashBalance>0.0</cashBalance><settledCashForInvestment>0.0</settledCashForInvestment><unSettledCashForInvestment>0.0</unSettledCashForInvestment><fundsWithheldFromPurchasePower>0.0</fundsWithheldFromPurchasePower><fundsWithheldFromWithdrawal>0.0</fundsWithheldFromWithdrawal><OpenCalls><cashCall>0.0</cashCall></OpenCalls><RealTimeValues><totalAccountValue>0.0</totalAccountValue><netMv>-454.22</netMv><netMvLong>0.0</netMvLong></RealTimeValues></Computed></BalanceResponse>'

In [101]:
positions = a_etrader.view_portfolio(accountIdKey)

In [102]:
positions[["symbolDescription", "pricePaid", "dateAcquired", "positionLotId", "positionId"]].head()

,symbolDescription,pricePaid,dateAcquired,positionLotId,positionId
0,USMV,43.640,1458532800000,9965198006,10227202005
1,IVV,210.130,1461643200000,15384288006,14848902005
2,IVV,495.500,1707800400000,2343150552103,14848902005
3,ITOT,47.105,1462161600000,16438625006,22411007005
4,ITOT,74.350,1596513600000,749658972103,22411007005


In [105]:
positions.set_index("symbolDescription").loc["USMV", "lotsDetails"]

'https://api.etrade.com/v1/accounts/E2ayfoKSzH1fFZs-7xV48A/portfolio/10227202005.json'

In [40]:
# view_portfolio_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"
view_portfolio_url = f"https://api.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"

In [57]:
viewp_resp = session.get(view_portfolio_url, params={"view": "QUICK", "lotsRequired": True})

In [58]:
data = viewp_resp.json()

In [59]:
ps_x = data["PortfolioResponse"]["AccountPortfolio"][0]["Position"]

In [61]:
data["PortfolioResponse"]["AccountPortfolio"][0]

{'accountId': '892405345',
 'Position': [{'positionId': 356478010900,
   'symbolDescription': 'AIRR',
   'dateAcquired': 1704430800000,
   'pricePaid': 55.1,
   'commissions': 0,
   'otherFees': 0,
   'quantity': 10,
   'positionIndicator': 'TYPE1',
   'positionType': 'LONG',
   'daysGain': 1.3,
   'daysGainPct': 0.2042,
   'marketValue': 637.7999,
   'totalCost': 551,
   'totalGain': 86.7999,
   'totalGainPct': 15.7531,
   'pctOfPortfolio': 4.6407,
   'costPerShare': 55.1,
   'todayCommissions': 0,
   'todayFees': 0,
   'todayPricePaid': 0,
   'todayQuantity': 0,
   'adjPrevClose': 63.65,
   'lotsDetails': 'https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/356478010900.json',
   'quoteDetails': 'https://api.etrade.com/v1/market/quote/AIRR.json',
   'Product': {'symbol': 'AIRR',
    'securityType': 'EQ',
    'securitySubType': 'ETF',
    'expiryYear': 0,
    'expiryMonth': 0,
    'expiryDay': 0,
    'strikePrice': 0,
    'productId': {'symbol': 'AIRR', 'typeCode': 'EQ

In [110]:
l_resp = session.get("https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/364189645900.json")
# l_resp = session.get('https://api.etrade.com/v1/accounts/E2ayfoKSzH1fFZs-7xV48A/portfolio/10227202005.json')

In [111]:
lot_data = l_resp.json()

In [112]:
lot_data

{'PositionLotsResponse': {'shortType': 1,
  'PositionLot': [{'positionId': 364189645900,
    'positionLotId': 2347253384103,
    'price': 475.0,
    'termCode': 2,
    'daysGain': -34.0,
    'daysGainPct': -7.7448,
    'marketValue': 405.0,
    'totalCost': 475.0,
    'totalCostForGainPct': -14.7368,
    'totalGain': -70.0,
    'lotSourceCode': 1,
    'originalQty': 1,
    'remainingQty': 1,
    'availableQty': 0,
    'orderNo': 99,
    'legNo': 1,
    'acquiredDate': 1707973200000,
    'locationCode': 1,
    'exchangeRate': 1.0,
    'settlementCurrency': 'USD',
    'paymentCurrency': 'USD',
    'adjPrice': 0.0,
    'commPerShare': 0.0,
    'feesPerShare': 0.0,
    'shortType': 1},
   {'positionId': 364189645900,
    'positionLotId': 2357680677103,
    'price': 444.0,
    'termCode': 2,
    'daysGain': -34.0,
    'daysGainPct': -7.7448,
    'marketValue': 405.0,
    'totalCost': 444.0,
    'totalCostForGainPct': -8.7837,
    'totalGain': -39.0,
    'lotSourceCode': 1,
    'originalQty'

In [60]:
[p for p in ps_x if p["symbolDescription"] == "MDB"]

[{'positionId': 364189645900,
  'symbolDescription': 'MDB',
  'dateAcquired': 1707973200000,
  'pricePaid': 459.5,
  'commissions': 0,
  'otherFees': 0,
  'quantity': 2,
  'positionIndicator': 'TYPE1',
  'positionType': 'LONG',
  'daysGain': -7.16,
  'daysGainPct': -0.7998,
  'marketValue': 888,
  'totalCost': 919,
  'totalGain': -31,
  'totalGainPct': -3.3732,
  'pctOfPortfolio': 6.4612,
  'costPerShare': 459.5,
  'todayCommissions': 0,
  'todayFees': 0,
  'todayPricePaid': 0,
  'todayQuantity': 0,
  'adjPrevClose': 447.58,
  'lotsDetails': 'https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/364189645900.json',
  'quoteDetails': 'https://api.etrade.com/v1/market/quote/MDB.json',
  'Product': {'symbol': 'MDB',
   'securityType': 'EQ',
   'expiryYear': 0,
   'expiryMonth': 0,
   'expiryDay': 0,
   'strikePrice': 0,
   'productId': {'symbol': 'MDB', 'typeCode': 'EQUITY'}},
  'Quick': {'lastTrade': 444.0,
   'lastTradeTime': 1709317408,
   'change': -3.58,
   'changePct':

In [24]:
data["PortfolioResponse"]["AccountPortfolio"][0]["totalPages"]

1

In [70]:
q_resp = session.get('https://api.etrade.com/v1/market/quote/MDB.json')

In [71]:
q_resp.json()

{'QuoteResponse': {'QuoteData': [{'dateTime': '17:15:49 EST 03-04-2024',
    'dateTimeUTC': 1709590549,
    'quoteStatus': 'CLOSING',
    'ahFlag': 'true',
    'hasMiniOptions': False,
    'All': {'adjustedFlag': False,
     'ask': 440.0,
     'askSize': 500,
     'askTime': '17:15:49 EST 03-04-2024',
     'bid': 435.3,
     'bidExchange': '',
     'bidSize': 2000,
     'bidTime': '17:15:49 EST 03-04-2024',
     'changeClose': 2.16,
     'changeClosePercentage': 0.49,
     'companyName': 'MONGODB INC CL A',
     'daysToExpiration': 0,
     'dirLast': '2',
     'dividend': 0.0,
     'eps': -2.63,
     'estEarnings': 2.924,
     'exDividendDate': 0,
     'high': 443.09,
     'high52': 509.62,
     'lastTrade': 439.0,
     'low': 427.29,
     'low52': 189.5901,
     'open': 437.08,
     'openInterest': 0,
     'optionStyle': '',
     'optionUnderlier': '',
     'previousClose': 436.84,
     'previousDayVolume': 1442620,
     'primaryExchange': 'NSDQ',
     'symbolDescription': 'MONGODB IN

In [30]:
import json

In [31]:
data = json.loads(viewp_resp.text)

In [102]:
viewp_resp.json() == data

True

In [86]:
datetime.fromtimestamp(1703048400000/1000)

datetime.datetime(2023, 12, 20, 5, 0)

In [ ]:
data

In [14]:
root_yr = Decimal("1")/Decimal("2.21095890411")

In [18]:
(Decimal("-0.118")+Decimal("1"))**root_yr+Decimal("-1")

Decimal('-0.0552087701011349359455755951')